In [1]:
import pandas as pd
import numpy as np

In [11]:
file_name = 'input.txt'
file = open(file_name)
data = file.read()
file.close()
data_list = data.split("\n")

In [82]:
#parsing output for full filepath
#Need to capture empty subdirectories, otherwise iterative directory size will not work for empty folders
curPath = ''
sysContents = []
for i in range(len(data_list)):
    if data_list[i][0:5] == '$ cd ':
        if data_list[i][5:7] == '..':
            curPath = curPath.rsplit('/',1)[0]
        else:
            curPath = curPath + '/' + data_list[i][5:]
    elif (data_list[i][0:4] != '$ ls'):
        if data_list[i][0:3] == 'dir':
            sysContents.append({'filePath':curPath,'fileName':'','fileSize':0})
        else:
            sysContents.append({'filePath':curPath,'fileName':data_list[i].split(' ',1)[1],'fileSize':int(data_list[i].split(' ',1)[0])})

In [83]:
#There is probably some way to avoid having to go from dict -> dataframe -> list -> dict...  but I'm dumb
dirSize = []
df = pd.DataFrame(sysContents)
dirList = df['filePath'].unique().tolist()
for i in range(len(dirList)):
    sz = 0
    for j in range(len(sysContents)):
        if sysContents[j]['filePath'][0:len(dirList[i])] == dirList[i]:
            sz = sz + sysContents[j]['fileSize']
    dirSize.append({'filePath':dirList[i],'fileSize':sz})

In [84]:
dirSize_df = pd.DataFrame(dirSize)

In [85]:
dirSize_df[dirSize_df['fileSize'] <= 100000]['fileSize'].sum()

1432936

In [112]:
neededSpace = 30000000 - 70000000 + dirSize_df[dirSize_df['filePath'] == '//']['fileSize'][0]

dirSize_df[dirSize_df['fileSize'] >= neededSpace].sort_values(by=['fileSize'])['fileSize'].iloc[0]

272298